In [ ]:
import pooch

url = f"https://github.com/LaPoGeoMar/Proj_Modelagem_Oleo/releases/download"
version = "v0.0.1.dev"

fname = pooch.retrieve(
    url=f"{url}/{version}/hycom-oil.nc",
    known_hash="sha256:dfc6f66055f19d43b189f6f4b4bec3397fb7536d61937d978004bc97f7860d99",
)

In [ ]:
import cf_xarray
import xarray as xr

ds = xr.open_dataset(fname)
ds

In [ ]:
ds.cf

In [ ]:
print(f"start: {ds['time'][0].to_numpy()}\nfinish: {ds['time'][-1].to_numpy()}")

In [ ]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

subset = ds.sel({"time": "2017-12-31", "depth": 0})

u = subset["water_u"]
v = subset["water_v"]

u_norm = u / np.sqrt(u**2.0 + v**2.0)
v_norm = v / np.sqrt(u**2.0 + v**2.0)

speed = (u**2 + v**2) ** 0.5

fig, ax = plt.subplots(figsize=(12, 12), subplot_kw={"projection": ccrs.PlateCarree()})


ax.contourf(subset["lon"], subset["lat"], speed)
ax.quiver(subset["lon"], subset["lat"], u_norm, v_norm, color="white", scale=50)

ax.coastlines(resolution="10m")

## Save a coast mask for later b/c we loose it when doing the averages below.

In [ ]:
coast_mask = np.ma.masked_invalid(u.to_numpy()).mask

## Season averages

In [ ]:
month_length = ds.time.dt.days_in_month

weights = (
    month_length.groupby("time.season") / month_length.groupby("time.season").sum()
)

# Test that the sum of the weights for each season is 1.0
np.testing.assert_allclose(weights.groupby("time.season").sum().values, np.ones(4))

# Calculate the weighted average
ds_weighted = (ds * weights).groupby("time.season").sum(dim="time")
ds_weighted

### Plot the season averages

In [ ]:
def plot_currents(ax, ds_weighted, season="DJF"):

    subset = ds_weighted.sel({"season": season, "depth": 0})

    u = subset["water_u"]
    v = subset["water_v"]

    u_norm = u / np.sqrt(u**2.0 + v**2.0)
    v_norm = v / np.sqrt(u**2.0 + v**2.0)

    speed = (u**2 + v**2) ** 0.5
    speed = np.ma.masked_array(speed, coast_mask)

    ax.contourf(subset["lon"], subset["lat"], speed)
    ax.quiver(subset["lon"], subset["lat"], u_norm, v_norm, color="white", scale=50)

    ax.coastlines(resolution="10m")

In [ ]:
fig, ((ax0, ax1), (ax2, ax3)) = plt.subplots(
    nrows=2, ncols=2, figsize=(25, 25), subplot_kw={"projection": ccrs.PlateCarree()}
)

ax0.set_title("Austral summer")
plot_currents(ax0, ds_weighted, season="DJF")

ax1.set_title("Austral fall")
plot_currents(ax1, ds_weighted, season="MAM")

ax2.set_title("Austral winter")
plot_currents(ax2, ds_weighted, season="JJA")

ax3.set_title("Austral spring")
plot_currents(ax3, ds_weighted, season="SON")